In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.2'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

!tar xf spark-3.3.2-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.3.2-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [15]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-04-13 18:18:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.70MB/s    in 0.2s    

2023-04-13 18:18:00 (5.70 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RemoteWorking").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [17]:
from pyspark import SparkFiles
# Load in respondents.csv from S3 into a DataFrame
url = "https://remoteworkbucket.s3.us-east-2.amazonaws.com/respondents.csv"
spark.sparkContext.addFile(url)

respondents_df = spark.read.option('header', 'true').csv(SparkFiles.get("respondents.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
respondents_df.show(10)

+------------+----------+------+--------------------+--------------------+-------------+--------------------+--------------------+-----------------+
|responder_id|birth_year|gender|       industry_desc|     occupation_desc|manage_others|           household|        years_at_job|metro_or_regional|
+------------+----------+------+--------------------+--------------------+-------------+--------------------+--------------------+-----------------+
|           0|      1972|Female|               Other|Clerical and admi...|           No|Couple with no de...|   More than 5 years|         Regional|
|           1|      1972|  Male|               Other|            Managers|          Yes|Couple with depen...|   More than 5 years|            Metro|
|           2|      1982|  Male|               Other|            Managers|          Yes|One  parent famil...|   More than 5 years|            Metro|
|           3|      1987|Female|Professional, Sci...|       Professionals|           No|Couple with depen.

In [19]:
#from pyspark import SparkFiles
# Load in orginizational_data.csv from S3 into a DataFrame
url = "https://remoteworkbucket.s3.us-east-2.amazonaws.com/orginizational_data.csv"
spark.sparkContext.addFile(url)

orginization_df = spark.read.option('header', 'true').csv(SparkFiles.get("orginizational_data.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
orginization_df.show(10)

+------------+------------------+--------------------+-------------------+-------------------+---------------------------+---------------------+---------------------------------+-------------------------------+----------------------------------------+
|responder_id| organization_size|   org_encouraged_rw|org_prepared_for_rw|rw_is_common_at_org|rw_permission_is_attainable|rw_collaboration_easy|if_no_covid_employer_encourage_rw|if_no_covid_employer_support_rw|if_no_covid_i_would_have_choice_about_rw|
+------------+------------------+--------------------+-------------------+-------------------+---------------------------+---------------------+---------------------------------+-------------------------------+----------------------------------------+
|           0|Between 20 and 199|      Strongly agree|     Somewhat agree|     Somewhat agree|             Strongly agree|       Somewhat agree|                Somewhat disagree|              Somewhat disagree|                       Strongly di

In [24]:
#from pyspark import SparkFiles
# Load in time_management.csv from S3 into a DataFrame
url = "https://remoteworkbucket.s3.us-east-2.amazonaws.com/time_management.csv"
spark.sparkContext.addFile(url)

time_df = spark.read.option('header', 'true').csv(SparkFiles.get("time_management.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
time_df.show(10)

+------------+------------------+----------------------------+------------------------------+-----------------------------+-------------------------+-----------------------+------------------------------------+----------------------------------------+----------------------+--------------------+---------------------------------+--------------------------------------+-----------------------+
|responder_id|rw_percentage_2020|preferred_rw_percentage_2020|preferred_rw_percentage_future|productivity_remote_vs_office|inperson_hours__commuting|inperson_hours__working|inperson_hours__personal_family_time|inperson_hours_domestic_responsibilities|remote_hours_commuting|remote_hours_working|remote_hours_personal_family_time|remote_hours_domestic_responsibilities|commute_time_difference|
+------------+------------------+----------------------------+------------------------------+-----------------------------+-------------------------+-----------------------+------------------------------------+----

In [21]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://remoteworking.ckzz8nmstjq2.us-east-2.rds.amazonaws.com:5432/RemoteWorking"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [22]:
#write respondents df into respondents table in rds
respondents_df.write.jdbc(url=jdbc_url, table= 'Respondents', mode=mode, properties= config)

In [23]:
#write orginizational df into orginizational data table in rds
orginization_df.write.jdbc(url=jdbc_url, table= 'Orginizational_Data', mode=mode, properties= config)

In [28]:
#write df orginizational into time management table in rds
time_df.write.jdbc(url=jdbc_url, table= 'Time_Management', mode=mode, properties= config)